<a href="https://colab.research.google.com/github/machine-perception-robotics-group/MPRGDeepLearningLectureNotebook/blob/master/12_gan/variational_autoencoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Variational autoencoder (VAE)

---
## 目的
Pytorchを用いてVariational autoencoder (VAE)を構築し，画像の再構成を行う．
また，潜在空間を可視化することで，VAEで獲得した表現を理解する．

※ VAEの理論について，本ノートブックの下部に記載しています．ご興味のある方はご確認ください．

## モジュールのインポート

はじめに必要となるモジュールをインポートします．

### GPUの確認
GPUを使用した計算が可能かどうかを確認します．

`GPU availability: True`と表示されれば，GPUを使用した計算を行うことが可能です．
Falseとなっている場合は，上部のメニューバーの「ランタイム」→「ランタイムのタイプを変更」からハードウェアアクセラレータをGPUにしてください．

In [ ]:
import os
from time import time
import numpy as np
from PIL import Image
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms

# GPUの確認
use_cuda = torch.cuda.is_available()
print('Use CUDA:', use_cuda)

## ネットワークの構築
VAEのネットワークを構築します．VAEは画像データを入力して特徴を抽出する**Encoder**と，任意のサイズのベクトルを入力して画像データを復元する**Decoder**で構築されています．

Decoderへ入力する潜在変数は，通常のautoencoderであればEncoderからの出力値を利用します．
一方で，VAEではEncodeした特徴量が任意の確率分布に従うように学習をします．
最も愚直な方法だとEncoderで抽出した特徴量を正規分布の平均$\mu$及び分散$\sigma$として扱うことで，Encoderの出力が正規分布に従うと考えられます．
Encoder，Decoderを個別に学習するのであれば，この方法でも問題ないです．
しかしながら，VAEはEncoderとDecoderを一貫して学習するため，このままだと中間層での微分が不可能です．
この問題点は，ニューラルネットワークを学習する際には非常に深刻な問題になります．
なぜなら，ニューラルネットワークは計算した誤差を逆伝播してネットワークのパラメータの更新量を決定することが必要不可欠であるため，ネットワーク全体を通して計算グラフがつながっている必要があるからです．

### Reparaterization trick
そこで，EncoderとDecoderの間に**Reparameterization trick**と呼ばれる処理を用いることで解決します．これは，Encoderが出力する特徴マップが正規分布に従うように学習するテクニックです．\
Reparameterization trickでは以下に示す式によって潜在変数$\bf{v}\in \mathbb{R}^{N}$をサンプリングします．
$$
\mathbf{v} = \mu + \exp\left(\frac{1}{2}\log\sigma\right)\odot\epsilon
$$
$$
s.t. \epsilon\leftarrow N(0, 1)
$$

### VAEのネットワーク構造
VAEの全体像のイメージを以下の図に示します．
Encoder及びDecoderは，全結合層とReLUを用いて構築します．平均$\mu$，分散$\sigma$はそれぞれ別の全結合層によって獲得します．ここで注意すべき点は，$\sigma$がReparametarization trickの計算過程で対数をとるため，$\sigma>0$でないと計算不可になることです．これを解決するために，厳密には全結合層が出力したベクトルを$\log\sigma$であると仮定して計算をしています．

<img src="https://dl.dropboxusercontent.com/s/efatumuniv5zdeq/vae.png" width=50%>

In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_dim=10):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
                nn.Linear(28*28, 256),
                nn.ReLU(inplace=True),
                nn.Linear(256, 100),
                nn.ReLU(inplace=True)
            )

        self.l_mu = nn.Linear(100, latent_dim)
        self.l_var = nn.Linear(100, latent_dim)

        self.decoder = nn.Sequential(
                nn.Linear(latent_dim, 100),
                nn.ReLU(inplace=True),
                nn.Linear(100, 256),
                nn.ReLU(inplace=True),
                nn.Linear(256, 28*28)
            )
    
    def reparameterization_trick(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        eps = torch.randn_like(std)
        latent = eps.mul(std).add_(mu)
        return latent
    
    def forward(self, x):
        h = self.encoder(x)
        mu = self.l_mu(h)
        logvar = self.l_var(h)
        latent = self.reparameterization_trick(mu, logvar)
        out = self.decoder(latent)
        return out, mu, logvar

## データセット，最適化関数などの設定
データセットはMNISTを用いて学習をします．
最適化関数にはAdam Optimizerを使用します．

In [ ]:
# データセットの設定
transform_train = transforms.Compose([transforms.ToTensor()])
mnist_data = datasets.MNIST(root='./data', train=True, transform=transform_train, download=True)
train_loader = DataLoader(dataset=mnist_data, batch_size=100, shuffle=True)

# ネットワークモデル・最適化手法の設定
model = VAE(latent_dim=2)
if use_cuda:
    model = model.cuda()    
optimizer = optim.Adam(model.parameters(), lr=1e-3)

## 誤差関数の設定
VAEの誤差関数を定義します．


VAEの誤差関数は以下のようになります（詳細はノートブック下部の「VAEの理論」参照）．
\begin{eqnarray}
\mathcal{L}(x,z) &=& \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{KL}[q(z|x)\|p(z)]
\end{eqnarray}
ここで，第1項は再構成誤差 (Reconstruction error)，第2項は正則化項 (Regularization error)と呼ばれます．

VAEの論文で再構成誤差は，負の対数尤度をとったベルヌーイ分布を仮定していますが，ここではBinary cross entropy (BCE) とKLダイバージェンスで定義します．
BCE lossは，Nをデータ数，xをネットワークの出力（ここでは出力画像），yを教師信号（ここでは入力画像）とすると，以下の式で表されます．
$$
\mathcal{L}_{bce} = -\sum_{i=1}^{N}y_{i}\log(x_{i}) + (1-y_{i})\log(1-x_{i})\\
$$
負の対数をとったベルヌーイ分布を展開すると，最終的にはBCEと同様の式になります．

また，潜在変数を標準正規分布へ近似するためのKLダイバージェンス$D_{KL}\left[N(\mu, \sigma)\|N(0, 1)\right]$は，以下の式で表現されます（詳細な展開は省略）．
$$
D_{KL}\left[N(\mu, \sigma)\|N(0, 1)\right] = \frac{1}{2}\sum_{i}(1+2\log \sigma_{i}-\mu_{i}^{2} - \sigma_{i}^{2})
$$
KLダイバージェンスは，ネットワーク全体の正則化の役割を果たします．

※ KLダイバージェンスは，分布間の距離を図る指標ですが，厳密な距離を表現していないことに注意してください．また，双方向性がないことにも注意してください．

この誤差関数の計算をPythonの関数`loss_function`として定義します．
`tilde_x`と`x`がそれぞれ，再構成した画像，入力画像のデータの引数，`mu`, `log_ver`がVAEの平均と分散を示しています．
`bce`はBCEを計算するためのpytorchのモジュールを入力するための引数です．

In [ ]:
def loss_function(tilde_x, x, mu, log_var, bce):
    reconstruction_loss= bce(tilde_x.view(-1, 784), x.view(-1, 784))
    kl_div = -0.5 * torch.sum(1 + log_var - mu.pow(2) - log_var.exp())
    return reconstruction_loss + kl_div

## ネットワークの学習

ネットワークの学習を行います．

まず，誤差計算に使用するBCE lossのPyTorchモジュール`BCEWithLogitsLoss`を定義します．
BCE lossを計算するためには，用いる値が[0, 1]の間で収まっている必要があります．
`BCEWithLogitsLoss`では，内部で入力された数値に対してSigmoid関数を適用して誤差計算を行なっているため，便宜上Decoderの出力値にはSigmoid関数を適用しないようなネットワーク構造を定義しています．

※ VAEの再構成誤差に対応するBCE lossは全体を平均しない代わりに，要素全てを合計する`reduction=='sum'`を使用しています．合計値を計算する誤差関数では，学習に使用する画像データの解像度と比例して誤差が増加することを覚えておいてください．
ここで平均値を計算する方法で誤差計算をおこなうと，KLダイバージェンスが負の値となる場合や，画像の復元ができなくなることがあるため，注意してください（詳細は割愛しますが，KLダイバージェンスが同時確率であることが理由です）．

学習回数`n_epoch`は10に設定し，学習を開始します．

In [ ]:
n_epoch = 10
bce = nn.BCEWithLogitsLoss(reduction='sum')
if use_cuda:
  bce = bce.cuda()

model.train()

start = time()
for epoch in range(1, n_epoch+1):
    sum_loss = 0.0
    for x, _ in train_loader:
        if use_cuda:
            x = x.cuda()
        optimizer.zero_grad()
        tilde_x, mu, log_var = model(x.view(x.size(0), -1))
        loss = loss_function(tilde_x, x, mu, log_var, bce)
        loss.backward()
        optimizer.step()

        sum_loss += loss.item()
        
    print("epoch:{}, mean loss: {}, elapsed time: {}".format(epoch,
                                                             sum_loss / len(train_loader),
                                                             time() - start))

## 学習済みモデルを用いて画像の復元
先ほど学習した重みパラメータを用いて，画像の復元をします．
まず，mnistのテストデータからランダムにサンプルした画像を入力した結果を確認します．

In [ ]:
test_transform = transforms.Compose([transforms.ToTensor()])
mnist_testdata = datasets.MNIST(root='./data', train=False, transform=test_transform)
test_loader =DataLoader(dataset=mnist_testdata, batch_size=10, shuffle=True)

model.eval()
with torch.no_grad():
    for idx, (x, y) in enumerate(test_loader):
        if use_cuda:
            x_in = x.cuda()
        y_in = y
        x_out, mu, logvar = model(x_in.view(x_in.size(0), -1))
        break

if use_cuda:
    x_in = x_in.cpu()
    x_out = x_out.cpu()

# 画像データを[0, 1] --> [0, 255]の値に変更，配列データの変換（numpy arrayへの変換）
output_img = (x_out*256.).clamp(min=0., max=255.).view(-1, 1, 28, 28).data.squeeze().numpy().astype(np.uint8)
input_img = (x_in * 256.).clamp(min=0., max=255.).data.squeeze().numpy()

fig = plt.figure(figsize=(10, 3))
# MNISTのテストデータ (上)
for i, im in enumerate(input_img):
    ax = fig.add_subplot(2, 10, i+1, xticks=[], yticks=[])
    ax.imshow(im, 'gray')

# VAEから出力された画像データ　（下）
for i, im in enumerate(output_img):
    ax = fig.add_subplot(2, 10, i+11, xticks=[], yticks=[])
    ax.imshow(im, 'gray')

## 潜在空間の可視化

学習で獲得した潜在空間の表現を可視化して確認します． 潜在空間の2次元ベクトルを擬似的に作成し，その値をDecoderへと入力し画像を生成することで，潜在空間の値を確認します．

※ この演算はhidden_num = 2の場合のAuto Encoderでのみ動作します．

In [ ]:
# 各次元のサンプリング点の生成
nv = 25
value1 = np.linspace(-2, 2, nv)
value2 = np.linspace(-2, 2, nv)

# 結果表示用のNumpy arrayの作成
plot_array = np.zeros([28 * 25, 28 * 25], dtype=np.float32)

# 潜在変数をDecoderへ入力し，画像を生成する
for i, yi in enumerate(value1):
    for j, xj in enumerate(value2):
        xx = torch.tensor([[yi, xj]], dtype=torch.float32)
        xx = xx.cuda()
        
        with torch.no_grad():
            output = model.decoder(xx)
            
        output = output.view(-1, 28, 28)
        output = output.cpu().numpy()
        plot_array[(25-i-1)*28:(25-i)*28, j*28:(j+1)*28] = (output*256.).clip(min=0., max=255.)

# 結果の表示
plt.figure(figsize=(10, 15))        
Xi, Yi = np.meshgrid(value1, value2)
plt.imshow(plot_array, origin="upper", cmap="gray")
plt.tight_layout()
plt.show()

# 課題

1. VAEの中間層のユニット数を変更して学習した際にどのような傾向が現れるか確認してみましょう．


# 参考文献
[1] Diederik P. Kingma and Max Welling, Auto-Encoding Variational Bayes, ICLR, 2014.\

## 画像分類と画像生成
機械学習をはじめとした画像分類モデルは，任意のデータ$x$をモデルに与えるとクラス確率を返すネットワークです．画像分類は任意のデータセットを用いて経験損失を最小化するようなモデルの重みパラメータを発見することが大きな目的です．\
一方，データ（画像や音声等）生成をするニューラルネットワークは，任意の潜在変数や過去の系列情報からデータ$p_{\theta}(x|z)$を生成することを目的としています．ただ，闇雲に潜在変数を与えているだけでは，求めたデータの生成が困難です．\
そのため，潜在変数から画像を生成するVAEでは，真のデータ分布$p(x)$を仮定した潜在空間から変数をサンプリングする戦略立てがされています．

## VAEの理論（なぜ変分推論？）
先に述べたように，VAEでは画像生成をするために真のデータ分布$p(x)$を利用する必要があります．しかしながら，我々がよく目にするデータセット達の分布は，複雑なので物理的に求めることが不可能です．\
では，ある潜在変数からデータを生成する$p_{\theta}(x|z)$を逆向きに$p_{\theta}(z|x)$として利用することで，実データの分布を探れそうですが，ニューラルネットワークの逆変換になるため計算が困難です．\
そこで，VAEでは実データの次元数を圧縮する$q_{\phi}(z|x)$を利用します．これによって，ニューラルネットワークを逆向きに利用する必要がなくなり計算が可能です．

### 変分下限
Encoder$q_{\phi}(z|x)$とDeocder$p_{\theta}(x|z)$を含むモデルを単に$p(x)$として，最尤推定により$x$にフィットするような最適なパラメータを発見することを考えます．\
しかしながら，積分の最大化問題を扱う必要があり，少々扱いづらいので変分下限を最大化することで下から抑えます．\
変分下限は以下の式変形によって得られます．
\begin{eqnarray}
\log p(x) &=& \log\int p(x,z)dz\\
&=& \log\int q(z|x)\frac{p(x, z)}{q(z|x)}\\
&\geq& \int q(z|x)\log\frac{p(x, z)}{q(z|x)}\\
&=& \mathcal{L}(x,z)
\end{eqnarray}
上の式変形の最後に出てきた$\mathcal{L}(x,z)$が変分下限となり，これを最大化すれば良いことになります．\
しかしながら，連続値に不等号が絡んでいるので厳密な最適値を導き出すことが困難で，僅かながらもギャップが生まれてしまいます．
従って，$p(x) - \mathcal{L}(x,z)$によってギャップを効率的に埋める術を知る必要があります．

### 変分下限のギャップ
$p(x) - \mathcal{L}(x,z)$のままでは先に進まないので，以下に式変形を示します．
\begin{eqnarray}
\log p(x) - \mathcal{L}(x,z) &=& \log p(x) - \int q(z|x)\log\frac{p(x,z)}{q(z|x)}dz\\
&=& \log p(x)\int q(z|x)dz - \int q(z|x)\log\frac{p(x,z)}{q(z|x)}dz\\
&=& \int q(z|x)\log p(x) dz - \int q(z|x)\log\frac{p(z|x)p(x)}{q(z|x)}dz\\
&=& \int q(z|x)\log p(x) dz - \int q(z|x)\left(\log p(z|x) + \log p(x) - \log q(z|x)\right)dz\\
&=& \int q(z|x)\left(\log p(x) - \log p(z|x) - \log p(x) \log q(z|x)\right)dz\\
&=& \int q(z|x)\log \frac{q(z|x)}{p(z|x)} dz\\ 
&=& D_{KL}[q(z|x)\|p(z|x)]
\end{eqnarray}
式変形の結果から，変分下限の素性が$\mathcal{L}(x,z)=\log p(x) - D_{KL}[q(z|x)\|p(z|x)]$だとわかりました．\
$\log p(x)$はされるため，非負のKLダイバージェンスを最小化すれば，変分下限を最大化することと同値です．

EncoderとDecoderそれぞれが出力する分布を近似すれば，真の分布を知ることができそうですが，Decoderを向きに利用している$p(z|x)$が含まれているので，計算が困難です．$D_{KL}[q(z|x)\|p(z|x)]$は綺麗な形にまとめれそうなので式変形をします．

### $D_{KL}[q(z|x)\|p(z|x)]$の式変形
ニューラルネットワークの逆変換が含まれている$D_{KL}[q(z|x)\|p(z|x)]$を式変形して美しい形にします．
\begin{eqnarray}
D_{KL}[q(z|x)\|p(z|x)] &=& \int q(z|x)\log\frac{q(z|x)}{p(z|x)}dz
\end{eqnarray}
ニューラルネットワークの逆変換である$p(z|x)$をなくすためにベイズの定理$p(z|x) = \frac{p(x|z)p(z)}{p(x)}$を利用して式変形を進めます．
\begin{eqnarray}
\int q(z|x)\log\frac{q(z|x)}{p(z|x)}dz &=& \int q(z|x)\left(\log q(z|x) - \log \frac{p(x|z)p(z)}{p(x)}\right)dz\\
&=& \int q(z|x)\left(\log q(z|x) - \log p(x|z) - \log p(z) + \log p(x) \right)dz\\
&=& \int q(z|x)\left(\log q(z|x) - \log p(x|z) - \log p(z)\right)dz + \log p(x)\\
&=& \int q(z|x)\left(\log \frac{q(z|x)}{p(z)}-\log p(x|z)\right)dz + \log p(x)\\
&=& \int q(z|x)\log \frac{q(z|x)}{p(z)}dz - \int q(z|x)\log p(x|z)dz + \log p(x)\\
&=& D_{KL}[q(z|x)\|p(z)] - \mathbb{E}_{q(z|x)}[\log p(x|z)] + \log p(x)
\end{eqnarray}
美しい形になったので，式変形をした結果を変分下限の式$\mathcal{L}(x,z)=\log p(x) - D_{KL}[q(z|x)\|p(z|x)]$に代入すると以下のように表すことができます．
\begin{eqnarray}
\mathcal{L}(x,z) &=& \log p(x) - D_{KL}[q(z|x)\|p(z)] + \mathbb{E}_{q(z|x)}[\log p(x|z)] - \log p(x)\\
&=& \mathbb{E}_{q(z|x)}[\log p(x|z)] - D_{KL}[q(z|x)\|p(z)]
\end{eqnarray}

これまでの式変形を踏まえると，変分下限はEncdoerに関するDecoderの期待値，つまり生成した画像と真のデータのエントロピーと，Encoderの出力する事後確率$q(z|x)$とユーザが任意に設定する事前確率$p(z)$を計算すれば良いことになります．
ここで，事前確率には確率分布の中でも比較的扱いが容易な標準正規分布が利用されることが多いです．